In [ ]:
from importnb import Notebook

with Notebook():
    import Acquisition_Traitement_Donnees  # Now Step1 is treated like a Python module
df=Acquisition_Traitement_Donnees.df
df

In [ ]:
import pandas as pd
# Ajout d'une colonne goal egale a 1 si goal 0 sinon
df['goal']=df['result'].apply(lambda x: 1 if x=='goal' else 0 )
#Separation des coordonnes en 2 colonnes x,y
# Assuming coordinates are in the format "(x, y)"
# Assuming coordinates are in the format "(x, y)"
# Assuming coordinates are in the format "(x, y)"
df[['x', 'y']] = df['coordinates'].str.extract(r'\(([^,]+),\s*([^)]+)\)').apply(pd.to_numeric, errors='coerce')
#Rassemble le nombre total de tir par type dans un dictionnaire
total_shots=df['shotType'].value_counts()
total_shots=total_shots.to_dict()
#Rassemble le nombre total de goal par type de tir dans un dictionnaire
goals = df[df['result'] == 'goal']['shotType'].value_counts()
goals=goals.to_dict()
df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#https://www.geeksforgeeks.org/bar-plot-in-matplotlib/
# Creation de la figure
fig = plt.subplots(figsize =(12, 8)) 

# Position des bars sur l'axe X
barWidth=0.3
bar =  np.arange(len(goals))
bar2 = [x + barWidth for x in bar]
colors = sns.color_palette("muted")  
# Creation des bars
#goals
plt.bar(bar, goals.values(),color =colors[0], width = 0.25, 
        edgecolor ='grey', label ='Goals') 
#total shots
plt.bar(bar2, total_shots.values(),color =colors[1], width = 0.25, 
        edgecolor ='grey', label ='Total shot')  

# Labels
plt.xlabel('Shot type', fontweight ='bold', fontsize = 15) 
plt.ylabel('Number of shots ', fontweight ='bold', fontsize = 15) 
#position des ticks et labels 
plt.xticks([r+barWidth/2 for r in range(len(goals))], goals.keys())
plt.title("Nombre de tirs et de buts par type de tir")
plt.legend()
plt.show() 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Creation de la figure
fig,ax = plt.subplots(figsize =(12, 8)) 

# Position des bars sur l'axe X
barWidth=0.3
br1 =  np.arange(len(goals))
br2 = [x + barWidth for x in br1]
colors = sns.color_palette("muted")  
# Plot
#goal
ax.bar(br1, goals.values(),color =colors[0], width = 0.25, 
        edgecolor ='grey', label ='Goals') 
#total shot
ax.bar(br2, total_shots.values(),color =colors[1], width = 0.25, 
        edgecolor ='grey', label ='Total shot')  
#Positionnement de pourcentage sur l'axe oppose
ax2=ax.twinx()
#Creation dun dict de pourcentage a partir des 2 autres dictionaires
pourcentage_but={k: goals[k]/total_shots[k] *100 for k in goals.keys()}
ax2.plot(br1, [pourcentage_but[k] for k in goals.keys()], color=colors[4], marker='o', linewidth=2, markersize=8, label='Goal Percentage')
plt.xlabel('Shot type', fontweight ='bold', fontsize = 15) 
ax.set_ylabel('Number of shots (log)', fontweight ='bold', fontsize = 15) 
#Log scale pour que ca soit plus visible
ax.set_yscale('log')
ax2.set_ylabel('Pourcentage', fontweight ='bold', fontsize = 15) 
plt.xticks([r+barWidth/2 for r in range(len(goals))], goals.keys())
#Ajout des legendes
lines1, labels1 = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
plt.legend(lines1+lines2,labels1+labels2)
plt.show() 

Asked Claude Sonnet 3.5 what would be a good way to plot data with the range of values of goals, I also provided him with the previous code 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Set figure size
fig, ax1 = plt.subplots(figsize=(15, 10))

# Set position of bar on X axis 
x = np.arange(len(goals))
width = 0.5

# Colors
colors = sns.color_palette("muted")

# Plot total shots bars
bars = ax1.bar(x, total_shots.values(), width, color=colors[1], edgecolor='grey', label='Total shots')

# Create secondary y-axis for goals
ax2 = ax1.twinx()

# Plot goals line
line = ax2.plot(x, goals.values(), color=colors[0], marker='o', linewidth=2, markersize=8, label='Goals')

# Set scales to logarithmic
#ax1.set_yscale('log')
ax2.set_yscale('log')

# Add value labels
for i, (shot, goal) in enumerate(zip(total_shots.values(), goals.values())):
    ax1.annotate(f'{shot}', (i, shot), xytext=(0, 15), textcoords='offset points', ha='center', va='bottom')
    ax2.annotate(f'{goal}', (i, goal), xytext=(0, -15), textcoords='offset points', ha='center', va='bottom')

# Set labels and title
ax1.set_xlabel('Shot type', fontweight='bold', fontsize=15)
ax1.set_ylabel('Number of shots (log)', fontweight='bold', fontsize=15)
ax2.set_ylabel('Number of goals (log)', fontweight='bold', fontsize=15)
plt.title('Comparison of Shot Types: Total Shots vs Goals', fontweight='bold', fontsize=18)

# Set x-ticks
plt.xticks(x, goals.keys(), rotation=45, ha='right')

# Add legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# Adjust layout and display
plt.tight_layout()
plt.show()

Separation des dictionnaires en 2 pour creer 2 sous figures

In [ ]:
firsthalfshots = list(total_shots.values())[:len(total_shots)//2 + 2]
secondhalfshots = list(total_shots.values())[len(total_shots)//2 +2:]
firstgoals = list(goals.values())[:len(goals)//2 + 2]
secondgoals = list(goals.values())[len(goals)//2 +2:]

fig, ax = plt.subplots(2, figsize=(10, 7))
space1 = np.arange(len(firsthalfshots))
space2 = np.arange(len(secondhalfshots))
colors = sns.color_palette("muted")
ax[0].bar(space1, firsthalfshots,label="goal",color=colors[1])
ax[0].bar(space1, firsthalfshots, bottom=firstgoals,label='Nombre de tir',color=colors[0])
ax[1].bar(space2, secondhalfshots,color=colors[1])
ax[1].bar(space2, secondhalfshots, bottom=secondgoals,color=colors[0])
ax[0].set_ylabel('buts marques')
ax[1].set_ylabel('buts marques')
fig.suptitle('Nombre de buts superposees sur le nombre de tirs divise en 2 sous figures')

ax[0].set_xticks(space1, list(total_shots.keys())[:len(total_shots)//2 + 2])
ax[1].set_xticks(space2, list(total_shots.keys())[len(total_shots)//2 +2:])
ax[0].legend(loc='upper right')
plt.tight_layout()
plt.show()


In [ ]:
# Define tiers
tier = len(total_shots) // 3
premiertier = list(total_shots.values())[:tier + 1]
secondtier = list(total_shots.values())[tier + 1:2*tier + 1]
thirdtier = list(total_shots.values())[2*tier + 1:]

firstgoals = list(goals.values())[:tier + 1]
secondgoals = list(goals.values())[tier + 1:2*tier + 1]
thirdgoals = list(goals.values())[2*tier + 1:]

fig, ax = plt.subplots(3, figsize=(10, 10))  # Create 3 subplots
space1 = np.arange(len(premiertier))
space2 = np.arange(len(secondtier))
space3 = np.arange(len(thirdtier))
colors = sns.color_palette("muted")

# First subplot
ax[0].bar(space1, firstgoals, label="Buts", color=colors[1])
ax[0].bar(space1, premiertier, bottom=firstgoals, label='Nombre de tirs', color=colors[0])
ax[0].set_ylabel('Buts marqués')
ax[0].set_title('Premier Tier')

# Second subplot
ax[1].bar(space2, secondgoals, label="Buts", color=colors[1])
ax[1].bar(space2, secondtier, bottom=secondgoals, label='Nombre de tirs', color=colors[0])
ax[1].set_ylabel('Buts marqués')
ax[1].set_title('Deuxième Tier')

# Third subplot
ax[2].bar(space3, thirdgoals, label="Buts", color=colors[1])
ax[2].bar(space3, thirdtier, bottom=thirdgoals, label='Nombre de tirs', color=colors[0])
ax[2].set_ylabel('Buts marqués')
ax[2].set_title('Troisième Tier')

# Set x-ticks for each subplot
ax[0].set_xticks(space1)
ax[0].set_xticklabels(list(total_shots.keys())[:tier + 1])
ax[1].set_xticks(space2)
ax[1].set_xticklabels(list(total_shots.keys())[tier + 1:2*tier + 1])
ax[2].set_xticks(space3)
ax[2].set_xticklabels(list(total_shots.keys())[2*tier + 1:])

# Place legend in the upper right of the first subplot
ax[0].legend(loc='upper right')

plt.tight_layout()
plt.show()


Relation distance but


In [ ]:
def goaldistance(x):
    """Retourne la distance euclidienne entre des coordonnees (x,y) et le but
        On assume les coordonnées du goal à (89,0) et (-89,0)
    Args:
        x (np.array): (x,y)

    Returns:
        _type_: float
    """
    if x[0] >= 0: #Verifie si le x est superieur a 0, base le tir sur ca 
        return np.linalg.norm(np.array(x) - np.array([89, 0]))
    else:
        return np.linalg.norm(np.array(x) - np.array([-89, 0]))

df['distance'] = df[['x', 'y']].apply(lambda x: goaldistance(x), axis=1)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
bins=np.arange(0,110,10)
df['distance_bins'] = pd.cut(df['distance'], bins=bins,include_lowest=True,right=True)
goal_ratio = df.groupby(['season', 'distance_bins'])['goal'].mean().reset_index()
fig,ax = plt.subplots(figsize =(12, 8)) 
colors = sns.color_palette("muted")  
#IL partage l'index X et les labels de distance
index_plot=goal_ratio[goal_ratio['season']==20182019]['goal'].index
label_distance=goal_ratio[goal_ratio['season']==20182019]['distance_bins']
ax.plot(index_plot,goal_ratio[goal_ratio['season']==20182019]['goal'],color =colors[0], marker='o', linewidth=2, markersize=8, label='2018-2019') 
ax.plot(index_plot,goal_ratio[goal_ratio['season']==20192020]['goal'],color =colors[1], marker='o', linewidth=2, markersize=8, label='2019-2020') 
ax.plot(index_plot,goal_ratio[goal_ratio['season']==20202021]['goal'],color =colors[2], marker='o', linewidth=2, markersize=8, label='2020-2021') 

# Ticks
ax.set_title('Taux de Buts en Fonction de la Distance au But par Saison (2018-2021)')  
plt.xlabel('Distance du but', fontweight ='bold', fontsize = 15) 
plt.ylabel('Goal ratio', fontweight ='bold', fontsize = 15) 
plt.xticks(index_plot, label_distance )
plt.legend()
plt.show() 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
bins=np.arange(0,110,10)
#On decoupe la distance en 10 categories 
df['distance_bins'] = pd.cut(df['distance'], bins=bins,include_lowest=True,right=True)

#On regroupe par saison et categorie de distance puis on fait la moyenne de goal ( vu que goal est un bool ca devrait donner le ratio (Source: GPT4))
goal_ratio = df.groupby(['season', 'distance_bins'])['goal'].mean().reset_index()
fig,ax = plt.subplots(3,figsize =(12, 8)) 

colors = sns.color_palette("muted")  
index_bar=goal_ratio[goal_ratio['season']==20182019]['goal'].index
label_distance=goal_ratio[goal_ratio['season']==20182019]['distance_bins']
ax[0].bar(index_bar,goal_ratio[goal_ratio['season']==20182019]['goal'],color =colors[0], label='2018-2019') 
ax[1].bar(index_bar,goal_ratio[goal_ratio['season']==20192020]['goal'],color =colors[1], label='2019-2020') 
ax[2].bar(index_bar,goal_ratio[goal_ratio['season']==20202021]['goal'],color =colors[2], label='2020-2021')


# Ticks  
plt.xlabel('Distance du but', fontweight ='bold', fontsize = 15) 
plt.ylabel('Goal ratio', fontweight ='bold', fontsize = 15) 
ax[0].set_xticks(index_bar, label_distance )
ax[1].set_xticks(index_bar,  label_distance)
ax[2].set_xticks(index_bar,  label_distance)
plt.legend()
plt.show() 

Premiere version avant que je decouvre pd.cut 


Question 3 : Heatmap


In [ ]:
#Ratio des goals par rapport a la distance et au type de tir
heat=df.groupby(['distance_bins','shotType'])['goal'].mean().reset_index()
heat['goal']=heat['goal']*100
#PIvot pour le heatmap
df_heat=heat.pivot_table(values='goal',index='shotType',columns='distance_bins')
df_heat.fillna(0,inplace=True)

In [ ]:
import seaborn as sns
ax=sns.heatmap(df_heat,annot=True,linewidths=0.5)
ax.set_title("Pourcentage de Buts en Fonction de la Distance et des Types de Tirs")

In [34]:
df.to_csv("../complexe.csv",index=False)